# create .txt files for frames to extract clean location

In [3]:
import os
import re
from PIL import Image, ImageEnhance, ImageFilter
import pytesseract

# Path to Tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

BASE_PATH = r'E:\JupyterNotebooks\Road_problem\Images_folder\raw_images'

def preprocess_image(image_path):
    try:
        img = Image.open(image_path)
    except Exception as e:
        print(f"Error opening image {image_path}: {e}")
        return None
    img = img.convert('L')
    img = ImageEnhance.Contrast(img).enhance(2)
    img = img.filter(ImageFilter.MedianFilter(size=3))
    return img

def extract_text(image_path):
    img = preprocess_image(image_path)
    if img is None:
        return ""
    return pytesseract.image_to_string(img)

def parse_gps(text):
    patterns = [
        r"Latitude[:=\s]*([-+]?[0-9]*\.?[0-9]+).*?Longitude[:=\s]*([-+]?[0-9]*\.?[0-9]+)",
        r"Lat[:=\s]*([-+]?[0-9]*\.?[0-9]+)[°]?\s*[,;\s]+Lon[g]?:?[:=\s]*([-+]?[0-9]*\.?[0-9]+)"
    ]
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
        if match:
            try:
                return float(match.group(1)), float(match.group(2))
            except:
                continue
    return None, None

def is_meaningful_line(line):
    if len(line) < 3:
        return False
    alnum_count = sum(c.isalnum() for c in line)
    total_chars = len(line.replace(" ", ""))
    if total_chars == 0:
        return False
    ratio = alnum_count / total_chars
    if (alnum_count >= 2 and ratio > 0.4) or 'Latitude:' in line or 'Longitude:' in line:
        if re.search(r'gps\s*map\s*camera|gps\s*han\s*camera', line, re.IGNORECASE):
            return False
        return True
    return False

def clean_ocr_text(text):
    cleaned_lines = []
    datetime_pattern = re.compile(
        r'^.*?\d{1,2}/\d{1,2}/\d{4}\s+\d{1,2}:\d{2}\s*(AM|PM|am|pm)?\s*GMT\s*[\+\-]?\d{0,2}:?\d{0,2}', re.IGNORECASE
    )

    for line in text.splitlines():
        line = line.strip()
        if not line:
            continue
        if datetime_pattern.match(line):
            continue
        if re.search(r'[@«:;#%~©]', line) and len(line) <= 30:
            continue
        if re.search(r'\bG[o0]+\s*g[l1]e\b', line, re.IGNORECASE):
            continue
        if line in ["GPS: Not found", "Location: Unknown"]:
            continue
        if re.match(r"Lat.*\d+.*Lon.*\d+", line, re.IGNORECASE):
            continue
        if is_meaningful_line(line):
            cleaned_lines.append(line)
    return '\n'.join(cleaned_lines).strip()

def process_images_in_folder(root_path):
    for dirpath, _, filenames in os.walk(root_path):
        for filename in filenames:
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(dirpath, filename)
                print(f"Processing: {image_path}")

                raw_text = extract_text(image_path)
                lat, lon = parse_gps(raw_text)
                cleaned_text = clean_ocr_text(raw_text)

                if lat is not None and lon is not None:
                    print(f"✅ Extracted GPS - Latitude: {lat}, Longitude: {lon}")
                else:
                    print("⚠️ GPS coordinates not found")

                txt_path = os.path.splitext(image_path)[0] + '.txt'
                with open(txt_path, 'w', encoding='utf-8') as f:
                    if cleaned_text:
                        f.write(cleaned_text + '\n')
                    f.write(f"Latitude: {lat if lat else 'Not found'}\n")
                    f.write(f"Longitude: {lon if lon else 'Not found'}\n")

if __name__ == "__main__":
    process_images_in_folder(BASE_PATH)


Processing: E:\JupyterNotebooks\Road_problem\Images_folder\raw_images\frame_00027.jpg
✅ Extracted GPS - Latitude: 19.056359, Longitude: 74.71248
Processing: E:\JupyterNotebooks\Road_problem\Images_folder\raw_images\frame_00028.jpg
✅ Extracted GPS - Latitude: 19.056368, Longitude: 74.71242
Processing: E:\JupyterNotebooks\Road_problem\Images_folder\raw_images\frame_00029.jpg
✅ Extracted GPS - Latitude: 19.056368, Longitude: 74.71242
Processing: E:\JupyterNotebooks\Road_problem\Images_folder\raw_images\frame_00030.jpg
✅ Extracted GPS - Latitude: 19.056368, Longitude: 74.71242
Processing: E:\JupyterNotebooks\Road_problem\Images_folder\raw_images\frame_00031.jpg
✅ Extracted GPS - Latitude: 19.056368, Longitude: 74.71242
Processing: E:\JupyterNotebooks\Road_problem\Images_folder\raw_images\frame_00032.jpg
✅ Extracted GPS - Latitude: 19.056368, Longitude: 74.71242
Processing: E:\JupyterNotebooks\Road_problem\Images_folder\raw_images\frame_00033.jpg
✅ Extracted GPS - Latitude: 19.056368, Longi

 <!-- complete script that:

Reads images from a folder

Extracts GPS (lat, lon) using OCR from each image

Reverse geocodes lat/lon to location name (using OpenStreetMap Nominatim)

Reads defect category from a label file (assumes labels stored separately)

Organizes images into Training Images/Location/Defect/ folders

Generates a CSV report counting images per Location and Defect -->



# Store data in csv format (frames, lat, lon, location, defect, count of defects)

In [1]:
import os
import csv
import re
from collections import defaultdict
from ultralytics import YOLO

# Paths
FRAMES_DIR = r'E:\JupyterNotebooks\Road_problem\Images_folder\raw_images'
YOLO_MODEL_PATH = r'E:\JupyterNotebooks\Road_problem\runs\detect\train\weights\best.pt'
OUTPUT_CSV = r'E:\JupyterNotebooks\Road_problem\Images_folder\detection_results_loc.csv'

# Load YOLO model
model = YOLO(YOLO_MODEL_PATH)

def parse_lat_lon(text):
    pattern = r"Latitude[:\s]*([-+]?[0-9]*\.?[0-9]+).*?Longitude[:\s]*([-+]?[0-9]*\.?[0-9]+)"
    match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
    if match:
        return float(match.group(1)), float(match.group(2))
    return None, None

def extract_address(text):
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    filtered = [line for line in lines if not re.search(r'\d{2}/\d{2}/\d{4}.*(AM|PM)', line)]
    address_lines = [line for line in filtered if not re.search(r'(latitude|longitude|gps|location)', line, re.IGNORECASE)]
    return ', '.join(address_lines) if address_lines else 'Unknown'

def detect_defects_yolo(image_path):
    results = model(image_path)
    defect_counts = defaultdict(int)

    for result in results:
        boxes = result.boxes
        class_ids = boxes.cls
        for cls_id in class_ids:
            cls_name = result.names[int(cls_id)]
            defect_counts[cls_name] += 1

    # Format: "Pothole: 2, Crack: 3"
    defect_summary = ', '.join(f"{cls}: {count}" for cls, count in defect_counts.items())
    total_count = sum(defect_counts.values())

    return defect_summary if defect_counts else 'None', total_count

def process_frames_and_text(frames_dir, output_csv):
    rows = []

    for file in os.listdir(frames_dir):
        if file.lower().endswith('.jpg'):
            img_path = os.path.join(frames_dir, file)
            txt_path = os.path.splitext(img_path)[0] + '.txt'

            if not os.path.exists(txt_path):
                continue

            with open(txt_path, 'r', encoding='utf-8') as f:
                text = f.read()

            lat, lon = parse_lat_lon(text)
            location = extract_address(text)
            defects_str, defect_count = detect_defects_yolo(img_path)

            rows.append({
                'Frame': file,
                'Latitude': lat if lat is not None else 'Not found',
                'Longitude': lon if lon is not None else 'Not found',
                'Location': location,
                'Defects': defects_str,
                'Defect Count': defect_count
            })

            print(f"{file} => {lat}, {lon} | {location} | Defects: {defects_str} | Count: {defect_count}")

    with open(output_csv, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['Frame', 'Latitude', 'Longitude', 'Location', 'Defects', 'Defect Count'])
        writer.writeheader()
        writer.writerows(rows)

    print(f"\n✅ CSV saved at: {output_csv}")

if __name__ == "__main__":
    process_frames_and_text(FRAMES_DIR, OUTPUT_CSV)



image 1/1 E:\JupyterNotebooks\Road_problem\Images_folder\raw_images\frame_00027.jpg: 480x640 1 pothole, 934.9ms
Speed: 10.5ms preprocess, 934.9ms inference, 22.8ms postprocess per image at shape (1, 3, 480, 640)
frame_00027.jpg => 19.056359, 74.71248 | elit, one, Ahmadnagar, Maharasntra, India, 3p47+g3g, Mohininagar, Kedgaon, Ahmadnagar,, Maharashtra 414005, India | Defects: pothole: 1 | Count: 1

image 1/1 E:\JupyterNotebooks\Road_problem\Images_folder\raw_images\frame_00028.jpg: 480x640 (no detections), 688.6ms
Speed: 4.4ms preprocess, 688.6ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)
frame_00028.jpg => 19.056368, 74.71242 | PROG |, 3p47+g3g, Mohininagar, Kedgaon, Ahmadnagar,, Maharashtra 414005, India | Defects: None | Count: 0

image 1/1 E:\JupyterNotebooks\Road_problem\Images_folder\raw_images\frame_00029.jpg: 480x640 (no detections), 549.6ms
Speed: 3.9ms preprocess, 549.6ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)
frame_00029.jpg =>

# count pothole, cracks, manhole

In [ ]:
import os
import csv
import re
from collections import defaultdict
from ultralytics import YOLO

# Paths
FRAMES_DIR = r'E:\JupyterNotebooks\Road_problem\Images_folder\raw_images'
YOLO_MODEL_PATH = r'E:\JupyterNotebooks\Road_problem\runs\detect\train\weights\best.pt'
OUTPUT_CSV = r'E:\JupyterNotebooks\Road_problem\Images_folder\detection_results_loc_selected_defect.csv'

# Load YOLO model
model = YOLO(YOLO_MODEL_PATH)

# Defects to include in the report
ALLOWED_DEFECTS = {"Pothole", "Crack", "Manhole"}

def parse_lat_lon(text):
    pattern = r"Latitude[:\s]*([-+]?[0-9]*\.?[0-9]+).*?Longitude[:\s]*([-+]?[0-9]*\.?[0-9]+)"
    match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
    if match:
        return float(match.group(1)), float(match.group(2))
    return None, None

def extract_address(text):
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    filtered = [line for line in lines if not re.search(r'\d{2}/\d{2}/\d{4}.*(AM|PM)', line)]
    address_lines = [line for line in filtered if not re.search(r'(latitude|longitude|gps|location)', line, re.IGNORECASE)]
    return ', '.join(address_lines) if address_lines else 'Unknown'

def detect_defects_yolo(image_path):
    results = model(image_path)
    defect_counts = defaultdict(int)

    for result in results:
        boxes = result.boxes
        class_ids = boxes.cls
        for cls_id in class_ids:
            cls_name = result.names[int(cls_id)]
            if cls_name in ALLOWED_DEFECTS:
                defect_counts[cls_name] += 1

    # Format: "Pothole: 2, Crack: 3"
    defect_summary = ', '.join(f"{cls}: {count}" for cls, count in defect_counts.items())
    total_count = sum(defect_counts.values())

    return defect_summary if defect_counts else 'None', total_count

def process_frames_and_text(frames_dir, output_csv):
    rows = []

    for file in os.listdir(frames_dir):
        if file.lower().endswith('.jpg'):
            img_path = os.path.join(frames_dir, file)
            txt_path = os.path.splitext(img_path)[0] + '.txt'

            if not os.path.exists(txt_path):
                continue

            with open(txt_path, 'r', encoding='utf-8') as f:
                text = f.read()

            lat, lon = parse_lat_lon(text)
            location = extract_address(text)
            defects_str, defect_count = detect_defects_yolo(img_path)

            rows.append({
                'Frame': file,
                'Latitude': lat if lat is not None else 'Not found',
                'Longitude': lon if lon is not None else 'Not found',
                'Location': location,
                'Defects': defects_str,
                'Defect Count': defect_count
            })

            print(f"{file} => {lat}, {lon} | {location} | Defects: {defects_str} | Count: {defect_count}")

    with open(output_csv, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['Frame', 'Latitude', 'Longitude', 'Location', 'Defects', 'Defect Count'])
        writer.writeheader()
        writer.writerows(rows)

    print(f"\n✅ CSV saved at: {output_csv}")

if __name__ == "__main__":
    process_frames_and_text(FRAMES_DIR, OUTPUT_CSV)



image 1/1 E:\JupyterNotebooks\Road_problem\Images_folder\raw_images\frame_00027.jpg: 480x640 1 pothole, 180.7ms
Speed: 4.1ms preprocess, 180.7ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
frame_00027.jpg => 19.056359, 74.71248 | elit, one, Ahmadnagar, Maharasntra, India, 3p47+g3g, Mohininagar, Kedgaon, Ahmadnagar,, Maharashtra 414005, India | Defects: None | Count: 0

image 1/1 E:\JupyterNotebooks\Road_problem\Images_folder\raw_images\frame_00028.jpg: 480x640 (no detections), 618.4ms
Speed: 3.2ms preprocess, 618.4ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)
frame_00028.jpg => 19.056368, 74.71242 | PROG |, 3p47+g3g, Mohininagar, Kedgaon, Ahmadnagar,, Maharashtra 414005, India | Defects: None | Count: 0

image 1/1 E:\JupyterNotebooks\Road_problem\Images_folder\raw_images\frame_00029.jpg: 480x640 (no detections), 633.1ms
Speed: 3.6ms preprocess, 633.1ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)
frame_00029.jpg => 19.0563

# code for count of overall defect in particular video, Store in csv

In [ ]:
import csv
from collections import Counter
from ultralytics import YOLO
import os

# === Config ===
video_path = r'D:\gps_vid\a.mp4'
model_path = r'E:\JupyterNotebooks\Road_problem\runs\detect\train\weights\best.pt'  # update with your model
output_csv_path = r'E:\JupyterNotebooks\Road_problem\Images_folder\event_counts.csv'  # desired output CSV file path

# === Run Inference on Video ===
model = YOLO(model_path)
results = model(video_path, save=True, stream=True)

# === Count Objects ===
event_counts = Counter()
for result in results:
    names = result.names
    for cls_id in result.boxes.cls:
        class_name = names[int(cls_id)]
        event_counts[class_name] += 1

# === Save to CSV ===
with open(output_csv_path, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Defect Type", "Count"])
    for defect, count in event_counts.items():
        writer.writerow([defect, count])

print(f"Event counts saved to {output_csv_path}")


# create sub folder for each defects present at each location

In [ ]:
import os
import shutil
import re
import time
from PIL import Image, ImageEnhance, ImageFilter, ImageDraw, ImageFont
import pytesseract
import requests
from ultralytics import YOLO

# === Config ===
RAW_IMAGES_PATH = r'E:\JupyterNotebooks\Road_problem\Images_folder\raw_images'
TRAINING_IMAGES_PATH = r'E:\JupyterNotebooks\Road_problem\Images_folder\train_images\location'
YOLO_MODEL_PATH = r'E:\JupyterNotebooks\Road_problem\runs\detect\train\weights\best.pt'

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Load YOLO model
model = YOLO(YOLO_MODEL_PATH)

def preprocess_image(image_path):
    img = Image.open(image_path).convert('L')
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(2)
    img = img.filter(ImageFilter.MedianFilter(size=3))
    return img

def extract_text(image_path):
    img = preprocess_image(image_path)
    return pytesseract.image_to_string(img)

def parse_gps(text):
    pattern = r"Lat[:\s]*([-+]?[0-9]*\.?[0-9]+)[°]?\s*[,;\s]+Lon[g]?:?\s*([-+]?[0-9]*\.?[0-9]+)"
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return float(match.group(1)), float(match.group(2))
    return None, None

def reverse_geocode(lat, lon):
    url = "https://nominatim.openstreetmap.org/reverse"
    params = {'format': 'json', 'lat': lat, 'lon': lon, 'zoom': 10, 'addressdetails': 1}
    try:
        response = requests.get(url, params=params, headers={'User-Agent': 'Mozilla/5.0'})
        if response.status_code == 200:
            data = response.json()
            addr = data.get('address', {})
            return addr.get('city') or addr.get('town') or addr.get('village') or addr.get('county') or "Unknown_Location"
    except Exception as e:
        print(f"Reverse geocode error: {e}")
    return "Unknown_Location"

def detect_defect(image_path):
    results = model(image_path, conf=0.15)  # More sensitive detection

    defects = []
    if results and results[0].boxes is not None:
        for box in results[0].boxes:
            cls_id = int(box.cls[0])
            label = model.names[cls_id]
            defects.append(label)

    print(f"Detected {len(defects)} object(s) in {os.path.basename(image_path)}: {defects}")

    if defects:
        # Return most frequent defect + results object
        return max(set(defects), key=defects.count), results[0]
    else:
        return "No_Defect", results[0]

def draw_boxes_on_image(image_path, results, save_path):
    image = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(image)

    font_path = r"C:/Windows/Fonts/Arial.ttf"
    font_size = 24
    font = ImageFont.truetype(font_path, font_size)

    if results.boxes is None or len(results.boxes) == 0:
        print(f"No bounding boxes to draw for {os.path.basename(image_path)}")
        return

    for box in results.boxes:
        xyxy = box.xyxy[0].tolist()  # [x1, y1, x2, y2]
        cls_id = int(box.cls[0])
        label = model.names[cls_id]
        conf = float(box.conf[0])

        # Draw rectangle around detected object
        draw.rectangle(xyxy, outline="red", width=2)

        text = f"{label} ({conf:.2f})"
        text_bbox = font.getbbox(text)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]

        # Draw filled background rectangle
        draw.rectangle(
            [xyxy[0], xyxy[1] - text_height, xyxy[0] + text_width, xyxy[1]],
            fill="red"
        )
        # Draw label text
        draw.text((xyxy[0], xyxy[1] - text_height), text, fill="white", font=font)

    image.save(save_path)
    print(f"✅ Saved image with bounding boxes: {os.path.basename(save_path)}")

def organize_images():
    for file in os.listdir(RAW_IMAGES_PATH):
        if not file.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        image_path = os.path.join(RAW_IMAGES_PATH, file)
        print(f"\n--- Processing {file} ---")

        # Extract GPS
        text = extract_text(image_path)
        lat, lon = parse_gps(text)
        if lat is None or lon is None:
            print(f"⚠️ GPS not found in {file}, skipping.")
            continue

        # Get location name from coordinates
        location = reverse_geocode(lat, lon)
        time.sleep(1)  # Respect API rate limits

        # Detect defect and get result
        defect, result = detect_defect(image_path)

        # Prepare output folder
        dest_dir = os.path.join(TRAINING_IMAGES_PATH, location, defect)
        os.makedirs(dest_dir, exist_ok=True)

        # Save only the image with bounding boxes
        boxed_img_path = os.path.join(dest_dir, os.path.splitext(file)[0] + "_boxed.jpg")
        draw_boxes_on_image(image_path, result, boxed_img_path)

if __name__ == "__main__":
    organize_images()


--- Processing frame_00027.jpg ---

image 1/1 E:\JupyterNotebooks\Road_problem\Images_folder\raw_images\frame_00027.jpg: 480x640 2 potholes, 319.6ms
Speed: 9.1ms preprocess, 319.6ms inference, 13.9ms postprocess per image at shape (1, 3, 480, 640)
Detected 2 object(s) in frame_00027.jpg: ['pothole', 'pothole']
✅ Saved image with bounding boxes: frame_00027_boxed.jpg

--- Processing frame_00028.jpg ---

image 1/1 E:\JupyterNotebooks\Road_problem\Images_folder\raw_images\frame_00028.jpg: 480x640 (no detections), 230.5ms
Speed: 4.2ms preprocess, 230.5ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)
Detected 0 object(s) in frame_00028.jpg: []
No bounding boxes to draw for frame_00028.jpg

--- Processing frame_00029.jpg ---

image 1/1 E:\JupyterNotebooks\Road_problem\Images_folder\raw_images\frame_00029.jpg: 480x640 1 pothole, 482.5ms
Speed: 13.4ms preprocess, 482.5ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
Detected 1 object(s) in frame_00029.jpg